In [25]:
#복잡한 데이터셋을 처리하기 위한 깊은 모델 구축해보기
#ResNet
#컨볼루션층의 출력에 전의 전 (2단계 전)계층에 쓰였던 입력을 더해줘서 특징이 유실되지않게함
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

EPOCHS = 300
BATCH_SIZE = 128

train_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./.data',train=True,download = True,transform=transforms.Compose([
    transforms.RandomCrop(32,padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])),batch_size=BATCH_SIZE,shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./.data',train=False,download = True,transform=transforms.Compose([
    transforms.RandomCrop(32,padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])),batch_size=BATCH_SIZE,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
#CNN을 깊게 쌓는 방법
# 인공신경망을 여러개 쌓는다고 해서 학습 성능이 계속 좋아지진 않는다, 최초 입력 이미지에 대한 정보가 소실되기 때문이다.
#ResNet은 이 문제를 해결하기 위하여 네트워크를 작은 블록인 Residual 블록으로 나눠서 사용한다.
#Residual 블록의 출력에 입력이였던 x를 더함으로 모델을 훨씬 더 깊게 할 수 있다.
#배치 정규화란, 학습률을 너무 높게 잡으면 기울기가 소실되거나 발산하는 증상을 예방하여 학습과정을 안정화하는 방법
#이 모델에서는 자체적으로 정규화를 수행하여 드롭아웃과 같은 효과를 냄

class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
#Sequential -> 여러 모듈 nn.Module 들을 하나로 묶어줘서 코드를 간결하게 할 수 있음

In [27]:
class ResNet(nn.Module):
    def __init__(self,num_classes = 10):
        super(ResNet,self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=1,padding=1,bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16,2,stride = 1)
        self.layer2 = self._make_layer(32,2,stride = 2)
        self.layer3 = self._make_layer(64,2,stride = 2)
        self.linear = nn.Linear(64,num_classes)
    def _make_layer(self,planes,num_block,stride):
        strides = [stride]+[1]*(num_block-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes,planes,stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out,8)
        out = out.view(out.size(0),-1)
        out = self.linear(out)
        return out
    


In [ ]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(),lr = 0.1,momentum=0.9,weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=50,gamma=0.1)
# 학습률 감소 기법 lr_scheduler 를 이용하여 학습이 진행함에 따라 학습률을 감소시켜(50번 반복당 0.1곱하기) 학습률을 조정함

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 1.4037, Accuracy: 50.14%
[2] Test Loss: 1.3798, Accuracy: 54.52%
[3] Test Loss: 0.8521, Accuracy: 70.18%
[4] Test Loss: 0.8813, Accuracy: 68.94%
[5] Test Loss: 0.8648, Accuracy: 70.36%
[6] Test Loss: 0.7899, Accuracy: 72.88%
[7] Test Loss: 0.9586, Accuracy: 68.21%
[8] Test Loss: 0.7940, Accuracy: 72.93%
[9] Test Loss: 0.8706, Accuracy: 70.76%
[10] Test Loss: 0.7360, Accuracy: 74.84%
[11] Test Loss: 0.7745, Accuracy: 73.86%
[12] Test Loss: 0.7296, Accuracy: 74.01%
[13] Test Loss: 0.7972, Accuracy: 73.26%
[14] Test Loss: 0.6730, Accuracy: 76.65%
[15] Test Loss: 0.8414, Accuracy: 71.31%
[16] Test Loss: 0.6636, Accuracy: 77.47%
[17] Test Loss: 0.7420, Accuracy: 74.80%
[18] Test Loss: 0.6589, Accuracy: 77.28%
[19] Test Loss: 0.6252, Accuracy: 78.69%
[20] Test Loss: 0.7091, Accuracy: 76.48%
[21] Test Loss: 0.5946, Accuracy: 79.52%
[22] Test Loss: 0.5949, Accuracy: 79.72%
[23] Test Loss: 0.6155, Accuracy: 79.21%
[24] Test Loss: 0.9510, Accuracy: 70.67%
[25] Test Loss: 0.7526, A